In [ ]:
%matplotlib inline
import sys, os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['THEANO_FLAGS'] = 'device=cpu,compiledir_format="ipynb_compiledir_%(platform)s-%(processor)s-%(python_version)s-%(python_bitwidth)s"'
sys.path.append(os.getcwd()+"/..")

from tasks import check as load_filename
from scripts.imgtovideo import imgs_to_video
from data import load_data


import matplotlib.pyplot as plt
import numpy as np

import theano
import theano.tensor as T

from lasagne import layers as L

from lasagnekit.misc.plot_weights import dispims_color, tile_raster_images

from IPython.display import HTML, Image

import pandas as pd

from tqdm import tqdm

import base64
import json

from skimage.io import imread, imsave
from skimage.transform import resize

def load_model(filename, **kw):

    model = load_filename(
        what="notebook", 
        filename=filename, 
        **kw
    )
    return model

def build_brush_func(layers):
    if 'biased_output' in layers:
        bias = layers['biased_output'].b.get_value()
    elif 'bias' in layers:
        bias = layers['bias'].b.get_value()
    else:
        bias = np.array(0.1)

    bias = bias[np.newaxis, np.newaxis, :, np.newaxis, np.newaxis]

    if 'scaled_output' in layers:
        scale = layers['scaled_output'].scales.get_value()
    elif 'scale' in layers:
        scale = layers['scale'].scales.get_value()
    else:
        scale = np.array((1.,))
    scale = scale[np.newaxis, np.newaxis, :, np.newaxis, np.newaxis]

    
    X = T.tensor4()

    B = L.get_output(layers['brush'], X)
    if len(layers['brush'].output_shape) == 4: # (ex, t, w, h)
        B = B.dimshuffle(0, 1, 'x', 2, 3)
    
    fn = theano.function(
        [X], 
        T.nnet.sigmoid(B * scale + bias)
    )
    return fn

def build_encode_func(layers):
    w = layers['output'].output_shape[2]
    X = T.tensor4()
    fn = theano.function(
        [X], 
        T.nnet.sigmoid(L.get_output(layers['coord'], X)[:, :, 0:2]) * w
    )
    return fn

def to_grid_of_images(seq_imgs, **kw):
    y = seq_imgs
    imgs = []
    for t in range(y.shape[1]):
        yy = y[:, t]
        if yy.shape[1] == 1:
            yy = yy[:, 0, :, :, np.newaxis] * np.ones((1, 1, 1, 3))
        else:
            yy = yy.transpose((0, 2, 3, 1))
        img = dispims_color(yy, **kw)
        imgs.append(img)
    return imgs

def seq_to_video(seq, filename='out.mp4', verbose=1, framerate=8, rate=8, **kw):
    # shape of seq should be : (examples, time, c, w, h)
    seq = to_grid_of_images(seq, **kw)
    seq = [np.zeros_like(seq[0])] + seq
    if os.path.exists(filename):
        os.remove(filename)
    imgs_to_video(seq, out=filename, verbose=verbose, framerate=framerate, rate=rate)

def embed_video(filename):
    video = open(filename, 'r+b').read()
    encoded = base64.b64encode(video)
    return HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                </video>'''.format(encoded.decode('ascii')))
def disp_grid(imgs, **kw):
    # shape of imgs should be : (examples, color, w, h)
    out = dispims_color(imgs.transpose((0, 2, 3, 1)) * np.ones((1, 1, 1, 3)), **kw)
    return out


def prop_uniques(x):
    x = x.reshape((x.shape[0], -1))
    x = map(hash_array, x)
    return len(set(x)) / float(len(x))

def hash_array(x):
    return hash(tuple(x))

def normalize(x, axis=1):
    return (x - x.min(axis=axis, keepdims=True)) / (x.max(axis=axis, keepdims=True) - x.min(axis=axis, keepdims=True))


## Load model and build functions

In [ ]:
# some selected models :
models = [
    "../training/brush7/model.pkl", #0 
    "../jobs/results/df2631577eaf38b002c6c0ae6f1937e1/model.pkl", #1
    "../jobs/results/68fa4141b9f970f6bde95da53b8e54fe/model.pkl", #2
    "../jobs/results/88b68a0c06bfb0a837cf7c0b77fe7eb9/model.pkl", #3
    "../jobs/results/9c70a6ee9340c85b398a31f7d16be962/model.pkl", #4
    "../jobs/results/bc8b5c4156561f4f09685874d52ba20e/model.pkl", #5
    "../jobs/results/a50c86acf5a2875dda6e1ac5b8d462c1/model.pkl", #6
    "../training/brush9/model.pkl", #7
    "../jobs/results/76f2a5320d2d700ee5943dceb2c3e004/model.pkl", #8
    "../jobs/results/b38bd0e4e6ac281e6eba581cbe9d62d3/model.pkl", #9
    "../training/brush12/model.pkl", #10
    "../jobs/results/8af0f0ff67a2c27a4bede27a9867c5cc/model.pkl", #11
    "../jobs/results/8fbec06424d9cae3887acf14bc01948b/model.pkl" #12
    "../training/brush16/model.pkl",#13
    "../jobs/results/cebca877494edddcdb140b5b8b481964/model.pkl", #14
    "../jobs/results/96b19e8ff6b43a3b0992f111ecafdb2c/model.pkl", #15
]

#0  : (mnist:nice,                   ir:emptyfixated,         strokes:bad)
#1  : (mnist:nice and curvy,         ir:noisy blobs,          strokes:very_bad and blurry)
#2  : (mnist:ok but a bit blurry,    ir:nice,                 strokes:bad and blurry)
#3  : (mnist:very nice but probs,    ir:noisy points,         strokes:bad but not blurry)
#4  : (mnist:very nice but blurBlo,  ir:emptyfixated,         strokes:bad but not blurry)
#5  : (mnist:very nice and squary,   ir:emptyfixated,         strokes:okayish)
#6  : (mnist:very nice and squary,   ir:emptyfixated,         strokes:okayish)
#7  : (mnist:very nice and squary,   ir:emptyfixated,         strokes:okayish)
#8  : (mnist:almostperfect,          ir:digits_fixatedone,    strokes:bad)
#9  : (mnist:almostperfect,          ir:digits_fixatedone,    strokes:bad)
#10 : (mnist:almostperfect,          ir:emptyfixated,         strokes:okayish)
#11 : (mnist:ok but a blurry,        ir:nice  but blobby,     strokes:bad and blurry)
#12 : (mnist:nice,                   ir:nice but fixated,     strokes:bad but not blurry)
#13 : (omniglot:nice,                ir:strokesfixated,       strokes:ok)
#14 : (omniglot:great,               ir:strokesfixated,       mnist:good!)
#15 : (aloi:get the shapes notdeta,  ir:needsnoisecool,       omniglot:funny)
filename = np.random.choice(models)
model, data, layers, w, h, c = load_model("../training/fractal/a5/model.pkl",
                                          force_w=16,
                                          force_h=16,
                                          dataset='cropped_digits',
                                          #force_model_params={'w_out': 64, 'h_out': 64},
                                          kw_load_data={'include_test': True})

In [ ]:
print(json.dumps(model.hypers['model_params'], indent=4))

In [ ]:
encode = build_encode_func(layers) # transforms image to sequence of coordinates

In [ ]:
brush = build_brush_func(layers) # transforms an image to sequence of images

In [ ]:
reconstruct = model.reconstruct # reconstructs an image

## Get coords from images

In [ ]:
encode(model.preprocess(data.X[0:10])).shape # (examples, time, 2)

## Create a video

In [ ]:
X = data.X[0:11*11]
#X = before_last + np.random.normal(0, 1, size=imgs[:, 0].shape).astype(np.float32) #(for colored images)
imgs = brush(model.preprocess(X)) # (examples, time, w, h)
#imgs = 1 - imgs
seq_to_video(imgs, 'seq.mp4')
embed_video('seq.mp4')

## Reconstruct

In [ ]:
im1 = disp_grid(model.preprocess(data.X[0:100]), border=1, bordercolor=(.3, .3, .3))
im2 = disp_grid(reconstruct(model.preprocess(data.X[0:100])), border=1, bordercolor=(.5, 0, 0))
im_mix = np.empty((im1.shape[0], im1.shape[1] + im2.shape[1], 3))
im_mix[:, 0:im1.shape[1]] = im1
im_mix[:, im1.shape[1]:] = im2
imsave('im_mix.png', im_mix)
Image('im_mix.png')

## iterative refinement

In [ ]:
np.random.seed(2)

nb_iter = 10
nb_examples = 100
thresh = 0.3
use_noise = False

# PREP
if use_noise: noise = np.random.normal(0, 0.5, size=imgs[:, 0].shape).astype(np.float32) #(for colored images)
else: noise = 0
if thresh == 'moving':
    whitepx_ratio = (data.X>0.5).sum() / np.ones_like(data.X).sum()
imgs = np.empty((nb_examples, nb_iter + 1, c, w, h)) # 1 = color channel
imgs = imgs.astype(np.float32)
#imgs[:, 0] = data.X[0:nb_examples].reshape((nb_examples, 1, h, w)) >0.5
imgs[:, 0] = np.random.uniform(size=(nb_examples, c, w, h))

scores = []
diversities = []

# ITERATIOn

for i in tqdm(range(1, nb_iter + 1)):
    
    if use_noise:noise = np.random.normal(0, 1, size=imgs[:, 0].shape).astype(np.float32) #(for colored images)
    else:noise = 0
        
    imgs[:, i] = brush(imgs[:, i - 1] + noise)[:,-1]
    if c == 1:
        if thresh == 'moving':
            vals = imgs[:, i].flatten()
            vals = vals[np.argsort(vals)]
            thresh_ = vals[-int(whitepx_ratio * len(vals)) - 1]
        else:
            thresh_ = thresh
        if thresh_:
            imgs[:, i] = imgs[:, i] > thresh_ # binarize
    score = np.abs(imgs[:, i - 1] - imgs[:, i]).sum()
    scores.append(score)
    diversity = prop_uniques(imgs[:, i])
    diversities.append(diversity)

In [ ]:
fig = plt.figure(figsize=(10, 10))
img = disp_grid(imgs[:, -1], border=1, bordercolor=(0.3, 0, 0))
imsave('ir.png', img)
Image('ir.png', width=500, height=500)

In [ ]:
seq_to_video(imgs, 'ir.mp4', border=0, bordercolor=(0, 0, 0))
embed_video('ir.mp4')

In [ ]:
plt.title('Score')
plt.plot(np.log(np.array(scores)) / np.log(10))
plt.show()
plt.title('Diversity')
plt.plot(diversities)
plt.show()

## Feeding a new image

In [ ]:
#dt_test = load_data('strokes', w=w,h=h) # for grayscale
#dt_test = load_data('digits', w=w, h=h)
dt_test = load_data('omniglot', w=w, h=h)

In [ ]:

# load from file

nb = 100
dt = dt_test.X[0:nb]
try:
    dt = dt.reshape((nb, c, w, h))
except Exception:
    dt = dt.reshape((nb, 1, w, h))
    dt = dt * np.ones((1, 3, 1, 1))
    dt = dt.astype(np.float32)
print(dt.shape)
rec = reconstruct(dt)
print(((rec - dt)**2).mean())

im1 = disp_grid(model.preprocess(dt[0:nb]), border=1, bordercolor=(.3, .3, .3))
im2 = disp_grid(reconstruct(model.preprocess(rec[0:nb])), border=1, bordercolor=(.5, 0, 0))
im_mix = np.empty((im1.shape[0], im1.shape[1] + im2.shape[1], 3))
im_mix[:, 0:im1.shape[1]] = im1
im_mix[:, im1.shape[1]:] = im2
imsave('im_mix_new_dataset.png', im_mix)
Image('im_mix_new_dataset.png')